https://medium.com/@somasunder/fine-tuning-bert-for-text-classification-a-step-by-step-guide-with-code-examples-0dea8513bcf2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import f1_score

In [ ]:
#data = pd.read_excel("/content/drive/MyDrive/Experiment2/expanded_annotator_dataset.xlsx")

#data=pd.read_csv("/content/drive/MyDrive/Experiment2/sentiment_analysis.csv")

#file_path="/content/drive/MyDrive/Experiment2/EXIST2023_Task1_en.xlsx"
#data = pd.read_excel(file_path)

# Encode labels into numeric format
#label_encoder = LabelEncoder()
#data["label_encoded"] = label_encoder.fit_transform(data["label_task1"])

# Split into training and validation sets
#train_texts, val_texts, train_labels, val_labels = train_test_split(data["tweet"].values, data["label_encoded"].values,test_size=0.2, random_state=42,)


In [ ]:
#file_path_d="EXIST_2023/Complete_disagreement_records.xlsx"
#file_path="/content/drive/MyDrive/Experiment2/female_disagreement_records.xlsx"
#file_path_m="/content/drive/MyDrive/Experiment2/male_disagreement_records.xlsx"

#file_path="/content/drive/MyDrive/Experiment2/age1_disagreement_records.xlsx"
#file_path="/content/drive/MyDrive/Experiment2/age2_disagreement_records.xlsx"
file_path="/content/drive/MyDrive/Experiment2/age3_disagreement_records.xlsx"
#file_path_a="EXIST_2023/Complete_agreement_records.xlsx"

#file_path = "EXIST_2023/EXIST2023_Task1_en.xlsx"
#data_d = pd.read_excel(file_path_d)
#data_f = pd.read_excel(file_path_f)
#data_m = pd.read_excel(file_path_m)
#data_a = pd.read_excel(file_path_a)

data = pd.read_excel(file_path)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data = data.dropna(subset=["tweet", "majority_vote"])

In [ ]:
import re

def remove_urls_and_lower(text):
    # Define the regex pattern for URLs starting with http or https
    url_pattern = re.compile(r'http[s]?://\S+')
    # Substitute the URLs with an empty string
    cleaned_text = url_pattern.sub('', text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text.strip()

In [ ]:
tweet_processed=data["tweet"].apply(remove_urls_and_lower)

In [ ]:
len(tweet_processed)

975

In [ ]:
len(data["majority_vote"])

975

In [ ]:

train_texts, val_texts, train_labels, val_labels = train_test_split(data["tweet"].values, data["majority_vote"].values,test_size=0.2, random_state=42,)
#train_texts, val_texts, train_labels, val_labels = train_test_split(data["tweet"].values, data["female_vote"].values,test_size=0.2, random_state=42,)
#train_texts, val_texts, train_labels, val_labels = train_test_split(data["tweet"].values, data["male_vote"].values,test_size=0.2, random_state=42,)

In [ ]:
print(len(train_texts), len(train_labels))
print(len(val_texts), len(val_labels))

780 780
195 195


Tokenizer and Dataset Class

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class ClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Define dataset
max_len = 128
train_dataset = ClassificationDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = ClassificationDataset(val_texts, val_labels, tokenizer, max_len)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Model setup

In [ ]:
# Model and Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = BertForSequenceClassification.from_pretrained( "bert-base-uncased",num_labels=len(label_encoder.classes_))
model = BertForSequenceClassification.from_pretrained( "bert-base-uncased",num_labels=2)
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training loop

In [ ]:
# Training Loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, leave=True)

    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch} Loss: {total_loss / len(train_loader)}")


Epoch 0: 100%|██████████| 49/49 [00:17<00:00,  2.82it/s, loss=0.633]


Epoch 0 Loss: 0.6884209367693687


Epoch 1: 100%|██████████| 49/49 [00:16<00:00,  2.94it/s, loss=0.612]


Epoch 1 Loss: 0.6029154037942692


Epoch 2: 100%|██████████| 49/49 [00:17<00:00,  2.88it/s, loss=0.163]

Epoch 2 Loss: 0.3116659865695603


Evaluation

In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0
all_predictions=[]
all_labels=[]

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)
        all_predictions.extend(predictions.cpu().numpy())  #this line from chatgpt
        all_labels.extend(labels.cpu().numpy())         #this line from chatgpt

accuracy = correct / total
print(f"Validation Accuracy: {accuracy:.4f}")

# Compute F1 Score (macro and micro)
f1_macro = f1_score(all_labels, all_predictions, average='macro') #this line from chatgpt


print(f"F1 Score (Macro): {f1_macro:.4f}")


Validation Accuracy: 0.6615
F1 Score (Macro): 0.6474


SHAP

In [ ]:
import shap

In [ ]:
x=int(input("Enter the index for explanation: "))
text_instance = train_texts[x]

Enter the index for explanation: 20


In [ ]:
from transformers import TextClassificationPipeline

In [ ]:
text_pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer,device=device)

Device set to use cuda


In [ ]:
shap_explainer=shap.Explainer(text_pipe)
shap_values=shap_explainer([text_instance])
#print("Shap_values",shap_values)
#print("shap_values[0]",shap_values[0])
shap.plots.text(shap_values)
#shap.plots.text(shap_values[0])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:14, 14.18s/it]               


https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html